In [92]:
import pandas as pd

df = pd.read_csv("googleplaystore.csv")

data = df[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Content Rating']].groupby(['App']).max().dropna().reset_index()
data["Installs"] = pd.to_numeric(data["Installs"].str.replace(",","").str.replace("+",""))
data["Reviews"] = pd.to_numeric(data["Reviews"])
data = data[data["Type"] == "Free"]

q = .8
pop = data[data["Installs"] > data["Installs"].quantile(q)]
pop = pop[pop["Reviews"] > pop["Reviews"].quantile(q)]
pop.sort_values(["Installs","Reviews"], ascending=False)

rev_scores = pop.sort_values(["Installs","Reviews"], ascending=False)
rev_scores = rev_scores.groupby(["Category"]).count().reset_index()

rev_scores["Score"] = round(rev_scores["App"]/min(rev_scores["App"]))

content_scores = pop.sort_values(["Installs","Reviews"], ascending=False)
content_scores = content_scores.groupby(["Content Rating"]).count().reset_index()

<ipython-input-92-711790734b99>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data["Installs"] = pd.to_numeric(data["Installs"].str.replace(",","").str.replace("+",""))


In [144]:
categories = data["Category"].sort_values().unique()
category_scoreboard = rev_scores.iloc[:,[0,1,-1]].sort_values(["App"], ascending=False)
contents = data["Content Rating"].unique()
content_scoreboard = content_scores.iloc[:,:2].sort_values(["App"], ascending=False)

In [145]:
import ipywidgets as widgets
from IPython.display import display

size = widgets.IntSlider(max = 110)
size_widget = widgets.VBox([widgets.Label("Size [Mb]"), size])
category = widgets.Dropdown(options=categories)
category_widget = widgets.VBox([widgets.Label("Category"), category])
freepaid_button = widgets.ToggleButtons(options=["Free", "Paid"])
content_widget = widgets.RadioButtons(options=contents, desciption="Content rating:")

display(size_widget, category_widget, freepaid_button, content_widget)

ToggleButtons(options=('Free', 'Paid'), value='Free')

RadioButtons(options=('Everyone', 'Mature 17+', 'Everyone 10+', 'Teen', 'Unrated', 'Adults only 18+'), value='…

In [194]:
category_score = round (float ( category_scoreboard[category_scoreboard["Category"] == category.value]["Score"] ), 2)
content_score = float ( content_scoreboard[content_scoreboard["Content Rating"] == content_widget.value]["App"] )
size_score = round ( 1/size.value , 2)  ## dodać scoreboard

if freepaid_button.value == "Free": 
    free_score = 61 # tutaj możnabyłoby przypisać zmienną z analiz
else:
    free_score = 1

score = round (( category_score + size_score + content_score ) * free_score , 2)

print("Size score:       ", size_score)
print("Category score:   ", category_score)
print("Free/paid score:  ", free_score)
print("Content score:    ", content_score)
print("--------------------------")
print("TOTAL SCORE:      ", score)

Size score:        0.02
Category score:    31.0
Free/paid score:   1
Content score:     176.0
--------------------------
TOTAL SCORE:       207.02
